<a href="https://colab.research.google.com/github/ChaitaliV/Location-Extractor/blob/main/flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ChaitaliV/Location-Extractor
!pip install -r Location-Extractor/requirements.txt

Cloning into 'Location-Extractor'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 16 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), 32.65 KiB | 2.51 MiB/s, done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.3/745.3 kB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.8 MB/s eta 0:00:00
 

In [2]:
!ngrok authtoken 2Oi7yxSmNSVx99JN4GB0VOJ65CL_QXcR2c9EqSaq82WLo5b

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
from flask import Flask,render_template,request
import sys
from flask_ngrok import run_with_ngrok
sys.path.append('Location-Extractor')

In [4]:
from class_final import LocationExtractor

Mounted at /content/drive


In [6]:
sentence = "I went to Paris and visited the Eiffel Tower."
location_extractor = LocationExtractor(sentence)
location_extractor.extract_location(sentence)

[' Eiffel ']

In [ ]:
app = Flask(__name__, template_folder='Location-Extractor/templates')
run_with_ngrok(app)   

@app.route("/")
def home():
    return render_template('frontpage.html')

@app.route('/process',methods=['GET','POST'])
def data():
    paragraph = request.form['paragraph']
    # Perform any necessary processing on the paragraph variable here
    return paragraph

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://3fee-34-125-144-92.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [21/Apr/2023 17:01:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Apr/2023 17:01:37] "GET /favicon.ico HTTP/1.1" 404 -


In [7]:
!pip install openai==0.10.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.7/156.7 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 23.4 MB/s eta 0:00:00
  Created wheel for openai: filename=openai-0.10.2-py3-none-any.whl size=168354 sha256=65363309f250d1b0e92ee1da05b8ac660ce7eaa7320369bd03ec2b792b2e6269
  Stored in directory: /root/.cache/pip/wheels/3a/86/d9/4192a102e46f5685bc01e116898fb6f43968a9f9dd184b9d5b
Successfully built openai


In [ ]:
app.run()